Réalisé par : 

    __ ABDOULAHI SALAHANNE Ahmed
    __ CONDE Alama
    __ KPATOUKPA Kpodjro

In [ ]:
import numpy as np
# Import sklearn 
import pandas as pd
import seaborn as sns  # Bibliothèque pour la visualisation des données


In [ ]:
data = pd.read_csv("tictactoe.csv")
print ( 'Forme avant de supprimer les valeurs en double :' ,data.shape)
# Suppression des lignes en double le cas échéant 
data = data.drop_duplicates () 
print ( 'Forme après suppression des valeurs en double :' ,  data.shape )
print("Base d'exemples: ")
data.head(15)


# i1 i2	i3	i4	i5	i6	i7	i8	i9:  représentent les cellules du board
# 0.2=> vide, 0.4 => IA et 0.8 => player
# a1 a2	a3	a4	a5	a6	a7	a8	a9 : représentent la place où jouer

In [ ]:
x = data.iloc[:, :9]
y = data.iloc[:, 9:]
x=x.to_numpy()
y=y.to_numpy()

In [ ]:
data.hist(["i1","i2","i3","i4","i5","i6","i7","i8","i9"],figsize=(18,10))

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pylab as plt

model = PCA(n_components=2)
model.fit(x)

In [ ]:
n_dims = x.shape[1]
model = PCA(n_components=n_dims)
model.fit(x)

variances = model.explained_variance_ratio_

meilleur_dims = np.argmax(np.cumsum(variances) > 0.95)


plt.bar(range(n_dims), np.cumsum(variances))
plt.hlines(0.95, 0, meilleur_dims, colors='r')
plt.vlines(meilleur_dims, 0, 0.95, colors='r')

In [ ]:
print(meilleur_dims+1)
# nombre de variables idéal pour l'apprentissage

In [ ]:
model = PCA(n_components=0.95)
model.fit(x)
xred=model.transform(x)
xred.shape

In [ ]:
class MultiLayerPerceptron:
    
    def __init__(self, arch , alpha = 0.1):
        # poids + biais
        self.W = {}
        self.B = {}
        
        # Taux d'adaptation
        self.alpha = alpha
        
        # Architecture :nbre de couches et nombre de neurones par couche
        self.arch = arch
        
        # Initialisation des poids: valeurs issues d'une distribution normale
        for i in np.arange(1,len(self.arch)):  
            # Poids
            w = np.random.randn(self.arch[i], self.arch[i-1])
            self.W[i] = w/np.sqrt(self.arch[i])
            # Bias
            b = np.random.randn(self.arch[i],1)
            self.B[i] = b/np.sqrt(self.arch[i])            
            
    def sigmoid(self, x):
        return 1.0/(1 + np.exp(-x))
    
    def dsigmoid(self, x): # x correspond ici à sigmoid(uj(t)), voir le cours
        return x * (1 - x)
    
     # Calcul et mémorisation de l'état de tous les neurones du réseau 
    def forward_pass(self, x):
        a = np.atleast_2d(x).T
        
        stats = {}
        stats[0] = a
        for layer in np.arange(1, len(self.arch)):
            a = self.sigmoid(np.dot(self.W[layer], a) + self.B[layer])
            stats[layer] = a
        return stats    
    
    # Sortie du réseau associée à une entrée X (les états des autres neurones ne sont pas mémorisés)
    def predict(self, X):
        a = np.atleast_2d(X).T
        for layer in np.arange(1, len(self.arch)):
            a = self.sigmoid(np.dot(self.W[layer], a) + self.B[layer])
        return a
    
    # Calcul de l'erreur quadratique moyenne
    def quadratic_loss(self, X, Y):
        
        Y = np.atleast_2d(Y).T
         
        predictions = self.predict(X)
        n = X.shape[0]
        
        loss = (1/n) * 0.5 * np.sum((predictions - Y) ** 2) 
        return loss 
    
    # Calcul des gradients locaux 
    def compute_gradient(self, x, y):
     
        L = len(self.arch) - 1 # indice de la couche de sortie 
        # Gradients
        Gw = {}
        Gb = {}
        A = self.forward_pass(x)
        # Les vecteurs delta  
        D = {}
        y = np.atleast_2d(y).T
        deltaL = (A[L] - y) * self.dsigmoid(A[L])
        D[L] = deltaL # Pour la sortie 
        
        # Calculer les vecteurs delta des autres couches en utilisants les vecteurs delta de la couche suivante
        for l in np.arange(L-1, 0, -1):
            D[l] = (self.W[l+1].T.dot(D[l+1])) * self.dsigmoid(A[l])
        for l in np.arange(L, 0, -1):
            Gb[l] = D[l]
            Gw[l] = D[l].dot(A[l-1].T)        
       
        return (Gw, Gb)
    
    # Mise à jour par rapport à l'erreur moyenne (relative à un bloc d'exemples)
    def update_with_bloc(self, bloc):
      
        m = len(bloc)
        # Gradients locaux
        GCw = {}
        GCb = {}
        # Initialiser à zeros 
        for i in np.arange(1,len(self.arch)):
            GCw[i] = np.zeros(self.W[i].shape)
            GCb[i] = np.zeros(self.B[i].shape)
            
        # Calcul des gradients
        for x, y in bloc:
            Gw, Gb = self.compute_gradient(x, y)
            for i in np.arange(1,len(self.arch)): 
                GCw[i] += Gw[i]
                GCb[i] += Gb[i]
                
        # Mettre à jour les poids 
        for l in np.arange(1,len(self.arch)):
            self.W[l] = self.W[l] - (self.alpha/m)*(GCw[l])
            self.B[l] = self.B[l] - (self.alpha/m)*(GCb[l])
    
    # Iteration: entrainement en utilisant tous les exemples, un bloc de taille bloc_size chaque fois
    def train(self, D, bloc_size):
        train_size = len(D)
        np.random.shuffle(D) # tirage au sort
        
        # Bloc d'exemples
        blocs = [D[k : k + bloc_size] for k in range(0, train_size, bloc_size)]
        
        for bloc in blocs: # Mise à jour suite au passage de chaque bloc
            self.update_with_bloc(bloc)
  
    # Apprentissage
    def fit(self, X, Y, bloc_size = 20, iterations = 10000, error_min = 0.001, displayPeriod = 5000):
     
        # Exemples avec X et Y Assemblés
        D = list(zip(X,Y))
        
        # Erreurs
        errors = [self.quadratic_loss(X,Y)]   # Erreur initiale    
        
        iter = 0
        print("Itération: {}-{}, Erreur: {:.6f}".format(iter, iterations,errors[iter]))
        while iter < iterations and errors[iter] > error_min: # Tour de boucle 
            
            self.train(D, bloc_size)  # Mettre à jour 
            errors.append(self.quadratic_loss(X,Y))         # Nouvelle erreur
          
            if (iter+1) % displayPeriod == 0:
                print("Itération: {}-{}, Error: {:.6f}".format(iter + 1, iterations,errors[iter]))
            iter += 1
        
        if errors[iter] < error_min: # Erreur inférieur à la valeur minimale
            print("Fin: erreur minimale atteinte : {:.6f}.", errors[iter])
        elif iter == iterations:
            print("Fin: nombre maximum d'itérations atteint.")
       
        return (errors, iter)

In [ ]:
# Initialisation et apprentissage
pmc = MultiLayerPerceptron(arch=[x.shape[1],30,30, 9], alpha=0.1)
print(x.shape, y.shape)
(errs, iter_fin) = pmc.fit(x,y, iterations=1000, bloc_size=10, error_min=0.001, displayPeriod=200)

In [ ]:
for i in range(1000,1010): 
    print(y[i])
    print(np.argmax(pmc.predict(x[i])))
    print(sum(pmc.predict(x[i])))

In [ ]:
def get_move(board):
    mutated = []
    for i in range(9):
        if board[i] == 0:
            mutated.append(0.2)
        elif board[i] == 1:
            mutated.append(0.8)
        elif board[i] == 2:  # 2 is computer
            mutated.append(0.4)
    probabilities = pmc.predict([mutated])
    #print(probabilities)
    heighest = 0
    index = 0
    for i in range(9):
        if probabilities[i] > heighest and board[i] == 0:
            index = i
            heighest = probabilities[i]
    print(index)
    return index


def print_board(board):
    print()
    print(board[0], "|", board[1], "|", board[2])
    print("---------")
    print(board[3], "|", board[4], "|", board[5])
    print("---------")
    print(board[6], "|", board[7], "|", board[8])


def did_win(board, player):
    return (board[0] == player and board[1] == player and board[2] == player) or (
                board[3] == player and board[4] == player and board[5] == player) or (
                       board[6] == player and board[7] == player and board[8] == player) or (
                       board[0] == player and board[3] == player and board[6] == player) or (
                       board[1] == player and board[4] == player and board[7] == player) or (
                       board[2] == player and board[5] == player and board[8] == player) or (
                       board[0] == player and board[4] == player and board[8] == player) or (
                       board[2] == player and board[4] == player and board[6] == player)


def move_not_valid(board, move):
    return move < 0 or move > 8 or board[move] != 0
#
#
# print(get_move([0, 0, 0, 0, 1, 0, 0, 0, 0]))
# print_board([0, 0, 0, 0, 1, 0, 0, 0, 0])


In [ ]:
playing = True

while playing:

    board = [0, 0, 0, 0, 0, 0, 0, 0, 0]

    current_player = int(input("voulez-commencer en premier? ( 1:oui ; 2:non): "))

    print_board(board)
    for i in range(9):
        if current_player == 1:
            move = -1
            while move_not_valid(board, move):
                move = int(input("Entrez votre choix: ")) - 1
            board[move] = 1
            current_player = 2
        else:
            move = get_move(board)
            board[move] = 2
            current_player = 1
        print_board(board)
        if did_win(board, 1):
            print("You won!")
            break
        elif did_win(board, 2):
            print("You lost!")
            break
        elif i == 8:
            print("Match nul!")
            break
    answer = input("Un nouveau jeu? (y/n): ")
    if answer != "y":
        playing = False